# Creating a Chatbot with Milvus

## Preparation

### Virtual environment and Python libraries

If you're reading this, you may have already followed the instruction to create a virtual environment and loaded the libraries required for this project.

If not, here they are:

Create a virtual environment:

In [ ]:
! python -m venv ./chatbot_venv

Source your new environment:

In [ ]:
! source chatbot_venv/bin/activate

Upgrade pip to the latest version:

In [ ]:
! pip install --upgrade pip

Install these libraries: pandas, langchain, towhee, unstructured, milvus, pymilvus, sentence_transformers, gradio, openai

In [ ]:
! pip install pandas langchain towhee unstructured milvus pymilvus sentence_transformers gradio openai

### Document store

You'll use Milvus to store both document chunks and their corresponding embeddings. This makes it easy to perform semantic retrieval via vector similarity.

**Note**: You can skip this section if you already have a Milvus server ready locally, or via Zilliz.

If you don't have a server, let's use [Milvus Lite](https://milvus.io/docs/milvus_lite.md), a lightweight version of Milvus that works seamlessly with Jupyter Notebook (or any other app that uses a client/server connection.)

In [ ]:
from milvus import default_server

# Start Milvus service
default_server.start()

In [ ]:
Use this block to stop the server if you need to:

In [ ]:
# Stop Milvus service
default_server.stop()

### Set Variables

Next, let's set a few variables.

- The URIs for Milvus and SQLite (which will act as a memory store for chat.)
- A flag to drop existing Milvus collections
- The embed model for processing text. Change this variable to experiement with different models
- Your Milvus collection name. 
- The size of your embedding arrays
- Your OpenAI key
- Path to SQLite storage

In [ ]:
import getpass
import os

MILVUS_URI = 'http://localhost:19530'
[MILVUS_HOST, MILVUS_PORT] = MILVUS_URI.split('://')[1].split(':')
DROP_EXIST = True
EMBED_MODEL = 'all-mpnet-base-v2'
COLLECTION_NAME = 'chatbot_demo'
DIM = 768

OPENAI_API_KEY = getpass.getpass('Enter your OpenAI API key: ')

# Clean up chat history from any previous runs
if os.path.exists('./sqlite.db'):
    os.remove('./sqlite.db')

Run this and respond to the prompt for your OpenAI key.

## Build Knowledge Base

Now you need a pipeline that can load documents and split them into smaller checks. Then take the chunks and use a pretrained model to extract text embedding for each doc chunk.

**Load & split documents**

This sample code block processes the Twowhee home page.

In [ ]:
from towhee import pipe, ops, DataCollection

pipe_load = (
    pipe.input('source')
        .map('source', 'doc', ops.text_loader())
        .flat_map('doc', 'doc_chunks', ops.text_splitter(chunk_size=300))
        .output('source', 'doc_chunks')
)
DataCollection(pipe_load('https://towhee.io')).show()

**Text to Embedding**

Using a pretrained model, you convert each text chunk into an embedding. These embeddings enable semantic search. As highluighted abovie, switch models by adjusting EMBED_MODEL to the name of another pretrained model supported by Sentence Transformers.

For example, convert the following doc piece to a text embedding:

> SOTA Models
>
> We provide 700+ pre-trained embedding models spanning 5 fields (CV, NLP, Multimodal, Audio, Medical), 15 tasks, and 140+ model architectures. These include BERT, CLIP, ViT, SwinTransformer, data2vec, etc.*

In [ ]:
pipe_embed = (
    pipe.input('doc_chunk')
        .map('doc_chunk', 'vec', ops.sentence_embedding.sbert(model_name=EMBED_MODEL))
        .map('vec', 'vec', ops.np_normalize())
        .output('doc_chunk', 'vec')
)

text = '''SOTA Models

We provide 700+ pre-trained embedding models spanning 5 fields (CV, NLP, Multimodal, Audio, Medical), 15 tasks, and 140+ model architectures.
These include BERT, CLIP, ViT, SwinTransformer, data2vec, etc.
'''

DataCollection(pipe_embed(text)).show()

**Insert data**

Now it's time to insert the text and embeddings into Milvus. First, you need a collection. This code creates one, and will destroy any existing collections if DROP_EXIST is true.

In [ ]:
from pymilvus import (
    connections, utility, Collection,
    CollectionSchema, FieldSchema, DataType
)


def create_collection(collection_name):
    connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
    
    has_collection = utility.has_collection(collection_name)
    
    if has_collection:
        collection = Collection(collection_name)
        if DROP_EXIST:
            collection.drop()
        else:
            return collection

    # Create collection
    fields = [
        FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=DIM),
        FieldSchema(name='text', dtype=DataType.VARCHAR, max_length=500)
    ]
    schema = CollectionSchema(
        fields=fields,
        description="Towhee demo",
        enable_dynamic_field=True
        )
    collection = Collection(name=collection_name, schema=schema)
    
    # Change index here if you want to accelerate search
    
    index_params = {
        'metric_type': 'IP',
        'index_type': 'IVF_FLAT',
        'params': {'nlist': 1024}
        }
    collection.create_index(
        field_name='embedding', 
        index_params=index_params
    )
    return collection

Next, you need a pipeline to create the collection and insert vectors and doc chunks into it.

In [ ]:
from towhee import pipe, ops, DataCollection

pipe_insert = (
    pipe.input('collection_name', 'vec', 'doc_chunk')
        .map(('collection_name', 'vec', 'doc_chunk'), 'mr',
             ops.ann_insert.osschat_milvus(host=MILVUS_HOST, port=MILVUS_PORT))
        .output('mr')
)

### Complete Insert Pipeline

When you assemble the operators above, you have an end-to-end pipeline that builds your knowledge base.

In [ ]:
from towhee import pipe, ops, DataCollection

load_data = (
    pipe.input('collection_name', 'source')
        .map('collection_name', 'collection', create_collection)
        .map('source', 'doc', ops.text_loader())
        .flat_map('doc', 'doc_chunk', ops.text_splitter(chunk_size=300))
        .map('doc_chunk', 'vec', ops.sentence_embedding.sbert(model_name=EMBED_MODEL))
        .map('vec', 'vec', ops.np_normalize())
        .map(('collection_name', 'vec', 'doc_chunk'), 'mr',
             ops.ann_insert.osschat_milvus(host=MILVUS_HOST, port=MILVUS_PORT))
        .output('mr')
)

Use the pipeline to build a knowledge base with the Wikipedia page for [Frodo Baggins](https://en.wikipedia.org/wiki/Frodo_Baggins).

In [ ]:
data_source = 'https://en.wikipedia.org/wiki/Frodo_Baggins'
mr = load_data(COLLECTION_NAME, data_source)

print('Doc chunks inserted:', len(mr.to_list()))

## Search Knowledgebase

In [ ]:
pipe_search = (
    pipe.input('collection_name', 'query')
        .map('query', 'query_vec', ops.sentence_embedding.sbert(model_name=EMBED_MODEL))
        .map('query_vec', 'query_vec', ops.np_normalize())
        .map(('collection_name', 'query_vec'), 'search_res',
             ops.ann_search.osschat_milvus(host=MILVUS_HOST,
                                           port=MILVUS_PORT,
                                           **{'metric_type': 'IP', 'limit': 3, 'output_fields': ['text']}))
        .flat_map('search_res', ('id', 'score', 'text'), lambda x: (x[0], x[1], x[2]))
        .output('query', 'text', 'score')
)

For example, search knowledge for the question "What is Towhee?":

In [ ]:
query = 'Who is Frodo Baggins?'
DataCollection(pipe_search(COLLECTION_NAME, query)).show()

## Chat History

You'll need to input chat history to the LLM to get effective responses. For this, SQLite is a good choice.

First, here's a function to retrieve chat history. It reuses the Milvus collection name for the SQLite table.

In [ ]:
pipe_get_history = (
    pipe.input('collection_name', 'session')
        .map(('collection_name', 'session'), 'history', ops.chat_message_histories.sql(method='get'))
        .output('collection_name', 'session', 'history')
)

And here's one to store it:

In [ ]:
pipe_add_history = (
    pipe.input('collection_name', 'session', 'question', 'answer')
        .map(('collection_name', 'session', 'question', 'answer'), 'history', ops.chat_message_histories.sql(method='add'))
        .output('history')
)

Let's test the two methods.

In [ ]:
answer = 'Test answer'
session_id = 'sess01'

pipe_add_history(COLLECTION_NAME, session_id, query, answer)

# Check history
pipe_get_history(COLLECTION_NAME, session_id).get_dict()['history']

## Query Pipeline for Milvus and the LLM

Now, let's use Milvus with the LLM to create the Chatbot.

In [ ]:
chat = (
    pipe.input('collection_name', 'query', 'session')
        .map('query', 'query_vec', ops.sentence_embedding.sbert(model_name=EMBED_MODEL))
        .map('query_vec', 'query_vec', ops.np_normalize())
        .map(('collection_name', 'query_vec'), 'search_res',
             ops.ann_search.osschat_milvus(host=MILVUS_HOST,
                                           port=MILVUS_PORT,
                                           **{'metric_type': 'IP', 'limit': 3, 'output_fields': ['text']}))
        .map('search_res', 'knowledge', lambda y: [x[2] for x in y])
        .map(('collection_name', 'session'), 'history', ops.chat_message_histories.sql(method='get'))
        .map(('query', 'knowledge', 'history'), 'messages', ops.prompt.question_answer())
        .map('messages', 'answer', ops.LLM.OpenAI(api_key=OPENAI_API_KEY,
                                                  model_name='gpt-3.5-turbo',
                                                  temperature=0.8))
        .map(('collection_name', 'session', 'query', 'answer'), 'new_history', ops.chat_message_histories.sql(method='add'))
        .output('query', 'history', 'answer', )
)

Give it a try with a new query:

In [ ]:
new_query = 'Where did Frodo take the ring?'
DataCollection(chat(COLLECTION_NAME, new_query, session_id)).show()

## Gradio Chat Service

Finally, let's run the bot with Gradio

In [ ]:
import uuid
import io

def create_session_id():
    uid = str(uuid.uuid4())
    suid = ''.join(uid.split('-'))
    return 'sess_' + suid


def respond(session, query):
    res = chat(COLLECTION_NAME, query, session).get_dict()
    answer = res['answer']
    response = res['history']
    response.append((query, answer))
    return response


And the Gradio code:

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    session_id = gr.State(create_session_id)

    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown('''## Chat''')
            conversation = gr.Chatbot(label='conversation').style(height=300)
            question = gr.Textbox(label='question', value=None)
    
            send_btn = gr.Button('Send Message')
            send_btn.click(
                fn=respond,
                inputs=[
                    session_id,
                    question
                ],
                outputs=conversation,
            )

demo.launch(server_name='127.0.0.1', server_port=8902)

In [ ]:
demo.close()